In [46]:
# 处理金总+二调直调工作量数据，并写入数据库（不包含销售建岗、黄工开发的RPA）
import pandas as pd
from rpa.ssc.hr.sap.export_103_t70 import export_103_t70
from rpa.ssc.hr.orm.session import Session
from rpa.ssc.hr.orm.tb_ods_hr_work_amount import TB_ODS_HR_WORK_AMOUNT

In [9]:
df = pd.read_csv('d:/12月份人员工作量汇总.csv', dtype=str)

In [17]:
df['obj_id'] = df['obj_id'].astype(str)
staff_ids = df['obj_id'].values.tolist()

In [19]:
lt = export_103_t70(None, staff_ids)
df2 = lt.to_dataframe()
df2.columns = ['obj_id', '姓名', 'begin_date', 'staff_rng', 'staff_srng', '人事子范围_文本', '人员组', '人员子组', 'org_id',
                          '组织机构', 'job_id', '职务（岗位）具体名称', 'pay_rng', 'pay_mrng', 'cost_center_id']
df2['begin_date'] = df2['begin_date'].apply(lambda v: v.replace('.', ''))

In [43]:
df3 = pd.merge(df, df2, how='left', on="obj_id")
df3 = df3.fillna('')

In [45]:
with Session() as s:
    for idx in df3.index:
        s.add(TB_ODS_HR_WORK_AMOUNT(statis_date = df3['statis_date'][idx],
                                    statis_time =df3['statis_time'][idx],
                                    staff_name = df3['staff_name'][idx],
                                    staff_funj = df3['funj'][idx],
                                    rpa_funj =  df3['rpa_funj'][idx],
                                    action_name = df3['action_name'][idx],
                                    service_id = df3['service_id'][idx],
                                    obj_type = 'P',
                                    obj_id = df3['obj_id'][idx],
                                    begin_date= df3['begin_date'][idx],
                                    staff_rng= df3['staff_rng'][idx],
                                    staff_srng= df3['staff_srng'][idx],
                                    org_id= df3['org_id'][idx],
                                    job_id= df3['job_id'][idx],
                                    pay_rng= df3['pay_rng'][idx],
                                    pay_mrng= df3['pay_mrng'][idx],
                                    cost_center_id= df3['cost_center_id'][idx],
                                    rpa_id = '',
                                    sr_no =  df3['sr_no'][idx],
                                    filename = '',
                                    create_time = df3['create_time'][idx],
                                    update_time = df3['update_time'][idx]
                                     ))